In [3]:
import umap
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint
from collections import defaultdict
import numpy as np
import ipdb
import time
import umap

URL = "https://boardgamegeek.com/boardgame/169786/scythe/credits"

URLS = [
    # URL,
    "https://boardgamegeek.com/boardgame/223750/hardback/credits",
    "https://boardgamegeek.com/boardgame/92415/skull/credits",
    # "https://boardgamegeek.com/boardgame/381356/diced-veggies/credits",
    # "https://boardgamegeek.com/boardgame/292487/10-gallon-tank/credits",
    # "https://boardgamegeek.com/boardgame/393343/smug-owls/credits",
    # "https://boardgamegeek.com/boardgame/405537/dropolter/credits",
    # "https://boardgamegeek.com/boardgame/178900/codenames/credits",
    # "https://boardgamegeek.com/boardgame/2655/hive/credits",
    # "https://boardgamegeek.com/boardgame/43022/yomi/credits",
    # "https://boardgamegeek.com/boardgame/261901/tokyo-highway/credits",
    # "https://boardgamegeek.com/boardgame/218129/catch-the-moon/credits",
    # "https://boardgamegeek.com/boardgame/262543/wavelength/credits",
    # "https://boardgamegeek.com/boardgame/302520/hues-and-cues/credits",
    # "https://boardgamegeek.com/boardgame/348096/kites/credits",
    # "https://boardgamegeek.com/boardgame/27225/bananagrams/credits",
    # "https://boardgamegeek.com/boardgame/429293/the-fellowship-of-the-ring-trick-taking-game/credits",
    # "https://boardgamegeek.com/boardgame/316113/buy-the-vote/credits",
    # "https://boardgamegeek.com/boardgame/98778/hanabi/credits",
    # "https://boardgamegeek.com/boardgame/67877/anomia/credits",
    # "https://boardgamegeek.com/boardgame/373185/cool-cool-cool/credits",
    # "https://boardgamegeek.com/boardgame/197405/tak/credits",
    # "https://boardgamegeek.com/boardgame/150999/valley-of-the-kings/credits",
    # "https://boardgamegeek.com/boardgame/421374/landmarks/credits",
    # "https://boardgamegeek.com/boardgame/420091/dadada/credits",
]

CREDITS_I_DO_NOT_CARE_ABOUT = set(["Alternate Names", "Year Released"])

CREDITS_I_DO_CARE_ABOUT_WITH_DUPES = [
    ["Designer", "Designers"],
    ["Solo Designer"],
    ["Artist", "Artists"],
    ["Publisher", "Publishers"],
    ["Developer"],
    ["Graphic Designer"],
    ["Sculptor"],
    ["Editor"],
    ["Writer"],
    ["Insert Designer"],
    ["Categories"],
    ["Mechanisms"],
    ["Family"],
]

CREDITS_I_DO_CARE_ABOUT_WITHOUT_DUPES = [
    "Designers",
    "Solo Designer",
    "Artists",
    "Publishers",
    "Developer",
    "Graphic Designer",
    "Sculptor",
    "Editor",
    "Writer",
    "Insert Designer",
    "Categories",
    "Mechanisms",
    "Family",
]

def get_credit_items(driver, url):
    driver.get(url)
    credit_dict = {}
    primary_name = None
    # li.outline-item ng-scope
    #     div.outline-item-title outline-item-title-lg
    #     div.outline-item-description
    #         div.ng-binding
    credit_items = driver.find_elements(By.CSS_SELECTOR, "li.outline-item")
    for credit_item in credit_items:
        section_name = credit_item.find_element(By.CSS_SELECTOR, "div.outline-item-title").text
        if section_name in CREDITS_I_DO_NOT_CARE_ABOUT:
            continue
        credit_dict[section_name] = []
        outline = credit_item.find_element(By.CSS_SELECTOR, "div.outline-item-description")
        elements = outline.find_elements(By.CLASS_NAME, "ng-binding")
        for element in elements:
            element_name = element.get_attribute('innerHTML')
            if section_name == 'Primary Name':
                primary_name = element_name
                break
            credit_dict[section_name].append(element_name)
    return primary_name, credit_dict

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
titles = []
credit_dicts = []
for url in URLS:
    primary_name, credit_dict = get_credit_items(driver, url)
    titles.append(primary_name)
    credit_dicts.append(credit_dict)
    time.sleep(3)

for credit_types in CREDITS_I_DO_CARE_ABOUT_WITH_DUPES:
    for credit_dict in credit_dicts:
        # Single not plural
        for credit_form in credit_types[:-1]:
            if credit_form in credit_dict:
                credit_dict[credit_types[-1]] = credit_dict[credit_form]
                del credit_dict[credit_form]

# Get sets of credits

unique_credits = defaultdict(set)
for credit_type in CREDITS_I_DO_CARE_ABOUT_WITHOUT_DUPES:
    for credit_dict in credit_dicts:
        unique_credits[credit_type].update(credit_dict[credit_type])

# Alphabetize credits and get counts
sorted_unique_credits = {}
num_credits = {}
for credit_type in CREDITS_I_DO_CARE_ABOUT_WITHOUT_DUPES:
    sorted_unique_credits[credit_type] = sorted(list(unique_credits[credit_type]))
    num_credits[credit_type] = len(unique_credits[credit_type])

# For each credit, create a numpy array...
new_array = np.transpose(np.array(titles))
for credit_type in CREDITS_I_DO_CARE_ABOUT_WITHOUT_DUPES:
    # Create zeroed array of the right size
    
    one_hot = np.zeros((len(URLS), num_credits[credit_type]))
    for ind, credit_dict in enumerate(credit_dicts):
        # ipdb.set_trace()
        # indexes = [sorted_unique_credits[credit_type].index(i) for i in credit_dict[credit_type]]
        indexes = []
        for enabled_dim in credit_dict[credit_type]:
            try:
                index = sorted_unique_credits[credit_type].index(enabled_dim)
                indexes.append(index)
            except ValueError:
                pass
        # b[ind][np.arange(num_credits[credit_type]), indexes] = 1
        
        np.put(one_hot[ind], indexes, 1)
        # print(f"num_credits[credit_type]={num_credits[credit_type]}")
        # print(f"sorted_unique_credits[credit_type]={sorted_unique_credits[credit_type]}")
        # print(f"credit_type={credit_type}")
        # print(f"credit_dict={credit_dict}")
        # print(f"indexes={indexes}")
        # print(f"b={b}")
    
    new_array = np.vstack((new_array, one_hot.T))    
pprint(new_array)
new_array = new_array.T
array_without_titles = new_array[:,1:]

reducer = umap.UMAP(random_state=42)
# embedding = reducer.fit_transform(new_X)
reducer.fit(array_without_titles)
embedding = reducer.transform(array_without_titles)
assert(np.all(embedding == reducer.embedding_))
# Create a dictionary which 

# Create list of dict(string->list) for each mechanism etc.
# For each key, get the set of all unique mechanisms etc.
# Alphabatize.
# Create a vector which is horizontally stacked.

WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x64b2c62d94e3 <unknown>
#1 0x64b2c6008c76 <unknown>
#2 0x64b2c602f757 <unknown>
#3 0x64b2c602e029 <unknown>
#4 0x64b2c606cccc <unknown>
#5 0x64b2c606c47f <unknown>
#6 0x64b2c6063de3 <unknown>
#7 0x64b2c60392dd <unknown>
#8 0x64b2c603a34e <unknown>
#9 0x64b2c62993e4 <unknown>
#10 0x64b2c629d3d7 <unknown>
#11 0x64b2c62a7b20 <unknown>
#12 0x64b2c629e023 <unknown>
#13 0x64b2c626c1aa <unknown>
#14 0x64b2c62c26b8 <unknown>
#15 0x64b2c62c2847 <unknown>
#16 0x64b2c62d2243 <unknown>
#17 0x7e516f28e609 start_thread
